<a href="https://colab.research.google.com/github/doorqman/quant.invest/blob/master/%E5%AE%A2%E8%A3%BD%E5%8C%96%E7%B7%9A%E4%B8%8A%E7%9C%8B%E7%9B%A4%E8%BB%9F%E9%AB%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#安裝Talib
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib

(Reading database ... 144405 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [ ]:
import pandas as pd
import pandas_datareader.data as web
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
#輸入股票代號
stockid=input("Please input stock ID : ")

Please input stock ID : 2330


In [ ]:
#透過 pandas_datareader 由 YAHOO 取得股價
stockid=stockid + ".TW"
df = web.DataReader(stockid,"yahoo",start="2019-01-01")
df.rename(columns={'Close':'close', 'Open':'open', 'High':'high', 'Low':'low', 'Volume':'volume'}, inplace=True)
df=df[-250:] #看最近一年
df

,high,low,open,close,volume,Adj Close
Date,,,,,,
2019-06-24,242.0,240.0,241.0,241.0,43300507.0,233.309204
2019-06-25,241.5,237.0,241.0,238.5,28559283.0,230.888992
2019-06-26,236.5,234.0,235.0,234.5,27607041.0,227.016632
2019-06-27,241.5,236.0,236.0,240.5,42007834.0,232.825165
2019-06-28,241.5,238.0,241.5,239.0,25962203.0,231.373047
...,...,...,...,...,...,...
2020-06-29,315.0,310.0,314.0,312.0,56580892.0,312.000000
2020-06-30,314.0,311.0,313.5,313.0,48991860.0,313.000000
2020-07-01,318.0,314.0,315.0,317.5,29532342.0,317.500000


In [ ]:
#劃出股價走勢圖
fig = px.line(df, x=df.index, y=df['close'],labels={'x': 'Date','y':'Close'},title= stockid + " 股價走勢圖")
fig.show()

In [ ]:
from talib import abstract

# 畫出均線
SMA=abstract.SMA(df)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['close'],
                    mode='lines',
                    name='Close Price'))
fig.add_trace(go.Scatter(x=df.index, y=SMA,
                    mode='lines',
                    name='SMA'))


fig.update_layout(title= stockid +" MA 均線 " ,
                  xaxis_title=" Date ",
                  yaxis_title=" Price")
fig.show()

In [ ]:
from plotly.subplots import make_subplots
# 畫出RSI
RSI=abstract.RSI(df , timeperiod=30)

fig = make_subplots(specs=[[{"secondary_y": True}]],)

fig.add_trace(go.Scatter(x=df.index, y=RSI,name="RSI",
                    mode='lines'),
                    secondary_y=False)

fig.add_trace(go.Scatter(x=df.index, y=df['close'],name="收盤價",
                    mode='lines'),
                    secondary_y=True)

fig.update_layout(title= stockid +" RSI " ,
                  xaxis_title=" Date ",
                  yaxis_title=" 收盤價 & RSI")

fig.show()

In [ ]:
STOCH = abstract.STOCH(df)
# 畫出 KD
fig = make_subplots()

fig.add_trace(go.Scatter(x=STOCH.index, y=STOCH['slowk'],name="K",
                    mode='lines'))

fig.add_trace(go.Scatter(x=STOCH.index, y=STOCH['slowd'],name="D",
                    mode='lines'))
fig.show()

In [ ]:
#彙整出客製化線上看盤軟體
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=4, cols=1,row_heights=[1.2,0.6,0.6,0.6])

fig.add_trace(
    go.Candlestick(
                   x=df.index,
                   open=df['open'],
                   high=df['high'],
                   low=df['low'],
                   close=df['close'],
                   increasing_line_color= 'red',
                   decreasing_line_color= 'green',
                   name="Candlestick",
                   ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df.index, y=SMA,
               mode='lines',
               name='SMA'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=df.index, y=df['volume'],name="Volume"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=df.index, y=RSI,name="RSI",mode='lines'),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=STOCH.index, y=STOCH['slowk'],name="K",mode='lines'),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(x=STOCH.index, y=STOCH['slowd'],name="D",mode='lines'),
    row=4, col=1
)

fig.update_layout(height=1000, width=1200,title_text="客製化線上看盤軟體 StockID : " + stockid,xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
#輸出成 HTML 檔案
fig.write_html(stockid+".html")